### Face Similarity Caculation
- Loading the extracted name from txt file
- Extract face images paired with the extracted name
- Caculate the face similarity between the face images in the test dataset and images paired with the extracted name in the training dataset. 

In [ ]:
import os
def extract_text (orig_file):
    articles_names=open(orig_file, 'r', encoding="utf-8")
    next(articles_names)
    lines = [line.strip() for line in articles_names]
    result=[]
    for i in range(len(lines)):
        orig_line=lines[i]
        segs = orig_line.split("\t")
        result.append((segs[0], segs[3].split(",")[0]))
    return result

In [ ]:
ar_name_list=extract_text("test_title_eng_name.tsv")

In [ ]:
import os
d='faces'
sub_directories = [o for o in os.listdir(d) if os.path.isdir(os.path.join(d,o))]
idx_name={}
name_idx={}
idx=1
for sub_dir in sub_directories:
    idx_name[sub_dir]='face_'+str(idx)
    name_idx['face_'+str(idx)] = sub_dir
    idx+=1
name_idx

In [ ]:
from deepface import DeepFace
cnt = 0
img_files = [f for f in os.listdir("test_img_face") if f.endswith('.jpg')]
ar_img_files={}
for ar_name in ar_name_list:
    if ar_name[1].strip() != 'NA' and ar_name[1] in idx_name:
        if os.path.exists(os.path.join("mapped_face", idx_name[ar_name[1]])):
            df_results=[]
            for img_file in img_files:
                img_path=os.path.join("test_img_face", img_file)
                df = DeepFace.find(img_path = img_path, db_path =os.path.join("mapped_face", idx_name[ar_name[1]]), model_name='Facenet', enforce_detection=False)
                if len(df)>0:
                    df_results.append((img_path,df['Facenet_cosine'].mean(), len(df)))
                else:
                    df_results.append((img_path,"NA", 0))
            ar_img_files[ar_name[0]]=df_results
            cnt+=1
            print(cnt, " name completed")

In [ ]:
f = open("train_img_similarity.txt", "a")
for key, v in ar_img_files.items():
    for item in v:
        result=key+"\t"+os.path.basename(item[0])+"\t"+str(item[1])+"\t"+str(item[2])+"\n"
        f.write(result)
f.close()